In [74]:
from Bio import SeqIO
from Bio import Entrez
from sklearn.cluster import KMeans
from shutil import copyfile
from urllib.request import urlopen
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster import hierarchy
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import nan_euclidean_distances
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier as rf #using
from sklearn.model_selection import StratifiedKFold #using
from xgboost import XGBClassifier #using
from sklearn.preprocessing import LabelEncoder #using
from joblib import Parallel, delayed #NOT
from sklearn.feature_selection import SelectFromModel #using
from sklearn.neighbors import KNeighborsClassifier as knn #using
from sklearn.naive_bayes import GaussianNB #using
from sklearn import svm #using
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay #using
import concurrent.futures #using
from matplotlib.backends.backend_pdf import PdfPages #using
import os
import glob
import time
import re
import pickle
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [16]:
meta=pd.read_csv("../results/ML/filt_biallelic_filtered_classified_meta_K2.csv", index_col=0)
meta

,Species_STR2,POP,Procedencia,Latitude,Longitude,CRS,ADMIX_Cluster1,ADMIX_Cluster2,STR_Cluster1,STR_Cluster2,MEAN_Cluster1,MEAN_Cluster2,Classification_K2
samples,,,,,,,,,,,,,
CPB114sample,hibrido,-,cativeiro,NaN,NaN,NaN,0.255415,0.744585,0.258,0.742,0.256707,0.743293,Hyb
CPB115sample,hibrido,-,cativeiro,NaN,NaN,NaN,0.241161,0.758839,0.244,0.756,0.242581,0.757420,Hyb
CPB128sample,libidinosus,lib_cat,cativeiro,NaN,NaN,NaN,0.999990,0.000010,0.999,0.001,0.999495,0.000505,Sp1
CPB129sample,flavius,CdI,cativeiro,NaN,NaN,NaN,0.000010,0.999990,0.000,1.000,0.000005,0.999995,Sp2
CPB132sample,flavius,Mixed,cativeiro,NaN,NaN,NaN,0.101914,0.898086,0.105,0.895,0.103457,0.896543,Hyb
...,...,...,...,...,...,...,...,...,...,...,...,...,...
DT03sample,libidinosus,lib_delta,livre,-2.804028,-41.866389,EPSG:4674,0.999990,0.000010,1.000,0.000,0.999995,0.000005,Sp1
DT06sample,libidinosus,lib_delta,livre,-2.846389,-41.830556,EPSG:4674,0.999990,0.000010,1.000,0.000,0.999995,0.000005,Sp1
DT8sample,libidinosus,-,livre,-7.929078,-44.195986,EPSG:4674,0.999990,0.000010,1.000,0.000,0.999995,0.000005,Sp1


In [17]:
meta["Classification_K2"].value_counts()

Classification_K2
Sp2    82
Hyb    52
Sp1    22
Name: count, dtype: int64

In [18]:
gen=pd.read_csv("../results/ML/filt_biallelic_filtered.genotype.csv", index_col=0)
gen

,loc10094_pos51,loc10104_pos11,loc10269_pos49,loc10436_pos97,loc10512_pos9,loc10623_pos107,loc10640_pos19,loc10663_pos5,loc10751_pos171,loc10789_pos175,...,loc9256_pos190,loc9292_pos20,loc9394_pos231,loc9407_pos55,loc9481_pos181,loc9500_pos93,loc9596_pos16,loc9781_pos170,loc9911_pos133,loc9991_pos107
CPB114sample,2,1,2,1,0,0,1,0,2,2,...,1,2,0,2,2,2,2,2,1,1
CPB115sample,1,1,1,1,1,1,2,0,2,2,...,2,2,1,1,2,2,2,2,2,2
CPB128sample,2,1,1,2,2,2,1,1,2,2,...,1,2,1,1,2,2,2,2,2,2
CPB129sample,0,0,1,2,2,0,2,0,2,2,...,2,2,0,2,2,2,2,2,2,2
CPB132sample,1,0,2,2,2,0,2,0,2,2,...,2,2,0,2,1,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
DT03sample,1,2,1,2,2,2,2,2,0,2,...,1,2,2,0,2,2,2,2,2,2
DT06sample,2,2,2,0,2,2,2,2,1,1,...,2,2,2,2,2,2,2,2,2,1
DT8sample,1,2,2,2,2,1,2,2,1,1,...,0,2,2,2,2,2,2,0,2,1
MN37431sample,1,0,2,2,2,2,2,1,2,1,...,0,1,1,0,2,2,2,0,1,1


In [19]:
meta.index.tolist() == gen.index.tolist()

True

In [20]:
k=5

In [21]:
skf=StratifiedKFold(n_splits=k, shuffle=False, random_state=None)

In [22]:
classes=meta.iloc[:,-1].values
print(classes)
classes=LabelEncoder().fit_transform(classes)
print(classes)


['Hyb' 'Hyb' 'Sp1' 'Sp2' 'Hyb' 'Hyb' 'Hyb' 'Sp2' 'Hyb' 'Hyb' 'Sp1' 'Sp1'
 'Hyb' 'Hyb' 'Hyb' 'Sp1' 'Hyb' 'Sp1' 'Sp1' 'Sp1' 'Hyb' 'Hyb' 'Hyb' 'Hyb'
 'Sp2' 'Sp2' 'Sp2' 'Hyb' 'Hyb' 'Sp1' 'Sp1' 'Sp1' 'Sp1' 'Sp1' 'Hyb' 'Hyb'
 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Sp1' 'Sp1' 'Hyb' 'Hyb' 'Sp2' 'Sp2' 'Sp2'
 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Hyb' 'Hyb' 'Hyb' 'Sp2' 'Hyb'
 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Hyb' 'Hyb' 'Hyb' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2'
 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2'
 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Hyb' 'Hyb'
 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Hyb' 'Hyb'
 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2'
 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2'
 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Hyb' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2' 'Sp2'
 'Hyb' 'Hyb' 'Sp1' 'Hyb' 'Hyb' 'Sp1' 'Sp1' 'Sp1' 'Sp1' 'Sp1' 'Sp1' 'Sp1']
[0 0 1 2 0 0 0 2 0 0 1 1 0 0 0 1 0 1 1 1 0 0 0 0 2

In [23]:
skf.get_n_splits(gen,classes)

5

In [24]:
test=skf.split(gen, classes)

In [25]:
#This loop adds information of each train/test in k-fold
d={}
for i,(train_index, test_index) in enumerate(test,start=1):
    train_x=gen.iloc[train_index].values
    test_x=gen.iloc[test_index].values
    train_y=classes[train_index]
    test_y=classes[test_index]
    d[("train_x"+format(i))]=train_x
    d[("train_y"+format(i))]=train_y
    d[("test_x"+format(i))]=test_x
    d[("test_y"+format(i))]=test_y

In [26]:
files=np.asarray(list(d.keys()))
files


array(['train_x1', 'train_y1', 'test_x1', 'test_y1', 'train_x2',
       'train_y2', 'test_x2', 'test_y2', 'train_x3', 'train_y3',
       'test_x3', 'test_y3', 'train_x4', 'train_y4', 'test_x4', 'test_y4',
       'train_x5', 'train_y5', 'test_x5', 'test_y5'], dtype='<U8')

In [27]:
for i in range(1,k+1):
    files_in_fold=files[np.char.endswith(files,format(i))]
    train_in_fold=files_in_fold[np.char.startswith(files_in_fold,"train")]
    test_in_fold=files_in_fold[np.char.startswith(files_in_fold,"test")]
    
    print("fold"+format(i))
    
    print(d[train_in_fold[0]])
    print("length="+format(len(d[train_in_fold[0]])))
    print(d[train_in_fold[1]])
    print("length="+format(len(d[train_in_fold[1]])))

    print(d[test_in_fold[0]])
    print("length="+format(len(d[test_in_fold[0]])))
    print(d[test_in_fold[1]])
    print("length="+format(len(d[test_in_fold[1]])))
        

fold1
[[2 2 2 ... 2 2 0]
 [2 0 1 ... 2 2 2]
 [2 1 0 ... 2 2 0]
 ...
 [1 2 2 ... 0 2 1]
 [1 0 2 ... 0 1 1]
 [1 1 1 ... 1 2 2]]
length=124
[1 1 1 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 2 2 2 0 0 0 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2
 2 0 0 1 0 0 1 1 1 1 1 1 1]
length=124
[[2 1 2 ... 2 1 1]
 [1 1 1 ... 2 2 2]
 [2 1 1 ... 2 2 2]
 ...
 [2 0 2 ... 2 0 2]
 [2 0 2 ... 2 1 1]
 [1 0 1 ... 2 0 2]]
length=32
[0 0 1 2 0 0 0 2 0 0 1 1 0 0 0 1 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
length=32
fold2
[[2 1 2 ... 2 1 1]
 [1 1 1 ... 2 2 2]
 [2 1 1 ... 2 2 2]
 ...
 [1 2 2 ... 0 2 1]
 [1 0 2 ... 0 1 1]
 [1 1 1 ... 1 2 2]]
length=125
[0 0 1 2 0 0 0 2 0 0 1 1 0 0 0 1 0 2 2 2 1 1 1 1 0 0 1 1 0 0 2 2 2 2 2 2 2
 2 2 2 0 0 0 2 0 2 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2
 2 2 0 0 1 0 0 1 1 1 1 1 1 1]
l

In [28]:
#Getting a model for each fold
def get_xgb_model(d, k):
    files=np.asarray(list(d.keys()))
    models = []
    for i in range(1,k+1):
        files_in_fold=files[np.char.endswith(files,format(i))]
        train_in_fold=files_in_fold[np.char.startswith(files_in_fold,"train")]
        test_in_fold=files_in_fold[np.char.startswith(files_in_fold,"test")]
        
        # instanciar o modelo XGBoost
        model = XGBClassifier()
        # chamar o fit para o modelo
        model.fit(d[train_in_fold[0]], d[train_in_fold[1]], verbose=False)
        # fazer previsões em cima do dataset de teste
        #predictions = model.predict(z_test)
        models.append(model)
    return models

In [29]:
#Getting a model for each fold
def get_rf_model(d, k):
    files=np.asarray(list(d.keys()))
    models = []
    for i in range(1,k+1):
        files_in_fold=files[np.char.endswith(files,format(i))]
        train_in_fold=files_in_fold[np.char.startswith(files_in_fold,"train")]
        test_in_fold=files_in_fold[np.char.startswith(files_in_fold,"test")]
        
        # instanciar o modelo XGBoost
        model = rf(n_estimators=300)
        # chamar o fit para o modelo
        model.fit(d[train_in_fold[0]], d[train_in_fold[1]])
        # fazer previsões em cima do dataset de teste
        #predictions = model.predict(z_test)
        models.append(model)
    return models

In [59]:
def process_fold_dt(i, dictionary):
    d={}
    files = np.asarray(list(dictionary.keys()))
    files_in_fold = files[np.char.endswith(files, format(i))]
    train_in_fold = files_in_fold[np.char.startswith(files_in_fold, "train")]
    test_in_fold = files_in_fold[np.char.startswith(files_in_fold, "test")]
    clf = tree.DecisionTreeClassifier()
    clf.fit(dictionary[train_in_fold[0]], dictionary[train_in_fold[1]])
    d[train_in_fold[0]]=clf
    return d
def process_fold_rf(i, dictionary):
    d={}
    files = np.asarray(list(dictionary.keys()))
    files_in_fold = files[np.char.endswith(files, format(i))]
    train_in_fold = files_in_fold[np.char.startswith(files_in_fold, "train")]
    test_in_fold = files_in_fold[np.char.startswith(files_in_fold, "test")]
    clf = rf(n_estimators=300)
    clf.fit(dictionary[train_in_fold[0]], dictionary[train_in_fold[1]])
    d[train_in_fold[0]]=clf
    return d
def process_fold_xgb(i,dictionary):
    d={}
    files = np.asarray(list(dictionary.keys()))
    files_in_fold = files[np.char.endswith(files, format(i))]
    train_in_fold = files_in_fold[np.char.startswith(files_in_fold, "train")]
    test_in_fold = files_in_fold[np.char.startswith(files_in_fold, "test")]
    model = XGBClassifier()
    model.fit(dictionary[train_in_fold[0]], dictionary[train_in_fold[1]], verbose=False)
    d[train_in_fold[0]]=model
    return d

def process_fold_knn(i,dictionary):
    d={}
    files = np.asarray(list(dictionary.keys()))
    files_in_fold = files[np.char.endswith(files, format(i))]
    train_in_fold = files_in_fold[np.char.startswith(files_in_fold, "train")]
    test_in_fold = files_in_fold[np.char.startswith(files_in_fold, "test")]
    model = knn()
    model.fit(dictionary[train_in_fold[0]], dictionary[train_in_fold[1]])
    d[train_in_fold[0]]=model
    return d

def process_fold_naiveb(i,dictionary):
    d={}
    files = np.asarray(list(dictionary.keys()))
    files_in_fold = files[np.char.endswith(files, format(i))]
    train_in_fold = files_in_fold[np.char.startswith(files_in_fold, "train")]
    test_in_fold = files_in_fold[np.char.startswith(files_in_fold, "test")]
    model = GaussianNB()
    model.fit(dictionary[train_in_fold[0]], dictionary[train_in_fold[1]])
    d[train_in_fold[0]]=model
    return d
def process_fold_svm(i,dictionary):
    d={}
    files = np.asarray(list(dictionary.keys()))
    files_in_fold = files[np.char.endswith(files, format(i))]
    train_in_fold = files_in_fold[np.char.startswith(files_in_fold, "train")]
    test_in_fold = files_in_fold[np.char.startswith(files_in_fold, "test")]
    model = svm.SVC(decision_function_shape='ovo')
    model.fit(dictionary[train_in_fold[0]], dictionary[train_in_fold[1]])
    d[train_in_fold[0]]=model
    return d

def get_xgb_model_parallel(dictionary, k):
    models = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures=[]
        for i in range(1,k+1):
            future = executor.submit(process_fold_xgb, i, dictionary)
            futures.append(future)
        
        
        # Retrieve the results
        for future in concurrent.futures.as_completed(futures):
            models.update(future.result())
    
    models=dict(sorted(models.items()))
    return models
    
def get_rf_model_parallel(dictionary, k):
    models = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures=[]
        for i in range(1,k+1):
            future = executor.submit(process_fold_rf, i, dictionary)
            futures.append(future)
        
        
        # Retrieve the results
        for future in concurrent.futures.as_completed(futures):
            models.update(future.result())
    
    models=dict(sorted(models.items()))
    return models
def get_dt_model_parallel(dictionary, k):
    models = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures=[]
        for i in range(1,k+1):
            future = executor.submit(process_fold_dt, i, dictionary)
            futures.append(future)
        
        
        # Retrieve the results
        for future in concurrent.futures.as_completed(futures):
            models.update(future.result())
    
    models=dict(sorted(models.items()))
    return models

def get_knn_model_parallel(dictionary, k):
    models = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures=[]
        for i in range(1,k+1):
            future = executor.submit(process_fold_knn, i, dictionary)
            futures.append(future)
        
        
        # Retrieve the results
        for future in concurrent.futures.as_completed(futures):
            models.update(future.result())
    
    models=dict(sorted(models.items()))
    return models
def get_naiveb_model_parallel(dictionary, k):
    models = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures=[]
        for i in range(1,k+1):
            future = executor.submit(process_fold_naiveb, i, dictionary)
            futures.append(future)
        
        
        # Retrieve the results
        for future in concurrent.futures.as_completed(futures):
            models.update(future.result())
    
    models=dict(sorted(models.items()))
    return models
def get_svm_model_parallel(dictionary, k):
    models = {}
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures=[]
        for i in range(1,k+1):
            future = executor.submit(process_fold_svm, i, dictionary)
            futures.append(future)
        
        
        # Retrieve the results
        for future in concurrent.futures.as_completed(futures):
            models.update(future.result())
    
    models=dict(sorted(models.items()))
    return models

In [29]:
# Generate correlation matrix
correlation_matrix = gen.corr()

# Set threshold for correlation
threshold = 0.6  # Example threshold

# Find highly correlated variables
highly_correlated_vars = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            highly_correlated_vars.add(colname)

# Print highly correlated variables
print("Highly correlated variables:", highly_correlated_vars)

# Drop highly correlated variables from DataFrame
df_filtered = gen.drop(columns=highly_correlated_vars)

# Display the filtered DataFrame
print("\nFiltered DataFrame:")
print(df_filtered)

Highly correlated variables: {'loc811_pos20', 'loc28168_pos80', 'loc394_pos32', 'loc8615_pos75', 'loc7174_pos90', 'loc46703_pos35', 'loc17705_pos10', 'loc40573_pos219', 'loc47239_pos22', 'loc9500_pos93', 'loc14171_pos214', 'loc39316_pos115', 'loc5911_pos23', 'loc23221_pos176', 'loc18393_pos141', 'loc35936_pos118', 'loc2769_pos120', 'loc31265_pos25', 'loc6584_pos1', 'loc13992_pos26', 'loc39754_pos103', 'loc4028_pos195', 'loc4435_pos61', 'loc8336_pos436', 'loc36203_pos0', 'loc27087_pos171', 'loc37336_pos4', 'loc4752_pos91', 'loc48666_pos30', 'loc39778_pos83', 'loc30527_pos33', 'loc21732_pos21', 'loc39423_pos114', 'loc9196_pos18', 'loc9991_pos107', 'loc42071_pos23', 'loc45969_pos194', 'loc24013_pos82', 'loc43845_pos65', 'loc2422_pos2', 'loc27495_pos74', 'loc4745_pos55', 'loc32124_pos46', 'loc46213_pos5', 'loc45973_pos33', 'loc3782_pos46', 'loc28655_pos34', 'loc11607_pos126', 'loc2458_pos50', 'loc31352_pos37', 'loc29843_pos14', 'loc39088_pos209', 'loc29042_pos8', 'loc9911_pos133', 'loc2016

In [42]:
#Running without parallelization
t1=get_xgb_model(d, 5)

In [52]:
t_knn=get_knn_model_parallel(d, 5)

In [56]:
t_nvb=get_naiveb_model_parallel(d,5)

In [60]:
t_svm=get_svm_model_parallel(d,5)

In [43]:
#Running with parallelization
t_xgb=get_xgb_model_parallel(d, 5)

In [35]:
t_rf=get_rf_model_parallel(d,5)

In [36]:
t_dt=get_dt_model_parallel(d,5)

In [87]:
importants=list(t2.values())[4].feature_importances_
snpnames=gen.columns
pd.DataFrame({'Feature':snpnames,'Importance':importants}).sort_values('Importance', ascending=False)

,Feature,Importance
542,loc9196_pos18,0.151307
452,loc46973_pos30,0.082081
308,loc359_pos200,0.073975
285,loc33845_pos126,0.067146
10,loc10843_pos0,0.047463
...,...,...
202,loc26684_pos62,0.000000
201,loc26667_pos72,0.000000
200,loc26580_pos4,0.000000
199,loc2648_pos87,0.000000


In [73]:
def save_image(filename): 
    
    # PdfPages is a wrapper around pdf  
    # file so there is no clash and create 
    # files with no error. 
    p = PdfPages(filename) 
      
    # get_fignums Return list of existing  
    # figure numbers 
    fig_nums = plt.get_fignums()   
    figs = [plt.figure(n) for n in fig_nums] 
      
    # iterating over the numbers in list 
    for fig in figs:  
        
        # and saving the files 
        fig.savefig(p, format='pdf')  
      
    # close the object 
    p.close()   

In [ ]:
for i in range(1,k+1):
    files_in_fold=files[np.char.endswith(files,format(i))]
    test_in_fold=files_in_fold[np.char.startswith(files_in_fold,"test")]
   # print(list(t_xgb.values())[i-1].score(d[test_in_fold[0]],d[test_in_fold[1]]))
    predictions = list(t_xgb.values())[i-1].predict(d[test_in_fold[0]])
    cm = confusion_matrix(d[test_in_fold[1]], predictions, labels=list(t_xgb.values())[i-1].classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(t_xgb.values())[i-1].classes_)
    disp.plot()
save_image("xgb.pdf")


In [ ]:
for i in range(1,k+1):
    files_in_fold=files[np.char.endswith(files,format(i))]
    test_in_fold=files_in_fold[np.char.startswith(files_in_fold,"test")]
    print(list(t_rf.values())[i-1].score(d[test_in_fold[0]],d[test_in_fold[1]]))
    predictions = list(t_rf.values())[i-1].predict(d[test_in_fold[0]])
    cm = confusion_matrix(d[test_in_fold[1]], predictions, labels=list(t_rf.values())[i-1].classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(t_rf.values())[i-1].classes_)
    disp.plot()
save_image("rf.pdf")

In [ ]:
for i in range(1,k+1):
    files_in_fold=files[np.char.endswith(files,format(i))]
    test_in_fold=files_in_fold[np.char.startswith(files_in_fold,"test")]
    print(list(t_dt.values())[i-1].score(d[test_in_fold[0]],d[test_in_fold[1]]))
    predictions = list(t_dt.values())[i-1].predict(d[test_in_fold[0]])
    cm = confusion_matrix(d[test_in_fold[1]], predictions, labels=list(t_dt.values())[i-1].classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(t_dt.values())[i-1].classes_)
    disp.plot()
save_image("dt.pdf")

In [ ]:
for i in range(1,k+1):
    files_in_fold=files[np.char.endswith(files,format(i))]
    test_in_fold=files_in_fold[np.char.startswith(files_in_fold,"test")]
    print(list(t_knn.values())[i-1].score(d[test_in_fold[0]],d[test_in_fold[1]]))
    predictions = list(t_knn.values())[i-1].predict(d[test_in_fold[0]])
    cm = confusion_matrix(d[test_in_fold[1]], predictions, labels=list(t_knn.values())[i-1].classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(t_knn.values())[i-1].classes_)
    disp.plot()
save_image("knn.pdf")

In [ ]:
for i in range(1,k+1):
    files_in_fold=files[np.char.endswith(files,format(i))]
    test_in_fold=files_in_fold[np.char.startswith(files_in_fold,"test")]
    print(list(t_nvb.values())[i-1].score(d[test_in_fold[0]],d[test_in_fold[1]]))
    predictions = list(t_nvb.values())[i-1].predict(d[test_in_fold[0]])
    cm = confusion_matrix(d[test_in_fold[1]], predictions, labels=list(t_nvb.values())[i-1].classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(t_nvb.values())[i-1].classes_)
    disp.plot()
save_image("nvb.pdf")

In [ ]:
for i in range(1,k+1):
    files_in_fold=files[np.char.endswith(files,format(i))]
    test_in_fold=files_in_fold[np.char.startswith(files_in_fold,"test")]
    print(list(t_svm.values())[i-1].score(d[test_in_fold[0]],d[test_in_fold[1]]))
    predictions = list(t_svm.values())[i-1].predict(d[test_in_fold[0]])
    cm = confusion_matrix(d[test_in_fold[1]], predictions, labels=list(t_svm.values())[i-1].classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(t_svm.values())[i-1].classes_)
    disp.plot()
save_image("svm.pdf")

In [24]:
len(test_x)

31

In [36]:
fold_4[0]

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93, 103, 104, 105, 106, 107, 108, 109, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 147, 148, 152, 153, 154, 155])

In [25]:
testarray=meta.iloc[fold_1[1]].Classification_K2.to_numpy()
testarray

array(['Hyb', 'Hyb', 'Sp1', 'Sp2', 'Hyb', 'Hyb', 'Hyb', 'Sp2', 'Hyb',
       'Hyb', 'Sp1', 'Sp1', 'Hyb', 'Hyb', 'Hyb', 'Sp1', 'Hyb', 'Sp2',
       'Sp2', 'Sp2', 'Sp2', 'Sp2', 'Sp2', 'Sp2', 'Sp2', 'Sp2', 'Sp2',
       'Sp2', 'Sp2', 'Sp2', 'Sp2', 'Sp2'], dtype=object)

In [ ]:
for i, (train_index, test_index) in enumerate(skf.split(gen, classes)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")